### 1. Distinct 2. Order By (default ASC)

In [ ]:
select distinct author_id as id
from view
where author_id = view_id
order by author_id;

### 1. Common Table Expression (CTE) 2. Sum with Case When...then..else...end (counting the if else) 3. Coalesce 4. nullif

In [ ]:
with t1 as(
select ad_id, sum(case when action in ('Clicked') then 1 else 0 end) as clicked
from ads
group by ad_id
)

, t2 as
(
Select ad_id as ad, sum(case when action in ('Clicked','Viewed') then 1 else 0 end) as total
from ads
group by ad_id
)

Select a.ad_id, coalesce(round((clicked+0.0)/nullif((total+0.0), 0)*100, 2),0) as ctr
from
(
select *
from t1 join t2
on t1.ad_id = t2.ad) a
order by ctr desc, ad_id;

### Having (only used with Group By and need aggregation function)

In [ ]:
Select actor_id, director_id
from actordirector
group by actor_id, director_id
having count(*)>=3;

### 1. Subquery 2. Between (with date) 3. between value1 and value2 = (a >= value1 AND a <= value2)

In [ ]:
Select d.product_id, round((sum(price*units)+0.00)/(sum(units)+0.00),2) as average_price
from(
Select *
from prices p inner join unitssold u
where u.purchase_date between p.start_date and p.end_date) d
group by d.product_id;

### 1. Lead(col, 1) over() and Lag 2. find neighboring seat is free

In [ ]:
​​Select seat_id
from(
select seat_id, free,
lead(free,1) over() as next,
lag(free,1) over() as prev
from cinema) a
where a.free=True and (next = True or prev=True)
order by seat_id;

### Union

In [ ]:
(Select '[0-5>' as bin, 
 sum(case when duration/60 < 5 then 1 else 0 end) as total from Sessions)
union
(Select '[5-10>' as bin, 
sum(case when ((duration/60 >= 5) and (duration/60 < 10)) then 1 else 0 end) as total from Sessions)
 union
(Select '[10-15>' as bin, 
 sum(case when ((duration/60 >= 10) and (duration/60 < 15)) then 1 else 0 end) as total from Sessions)
 union
(Select '15 or more' as bin, 
 sum(case when duration/60 >= 15 then 1 else 0 end) as total from Sessions);


### Select the 21th with Limit and Offset

In [ ]:
SELECT
    *
FROM
    mytable
ORDER BY
    somefield
LIMIT 1 OFFSET 20;

### Where [colA] not in [colB]

In [ ]:
SELECT Name AS Customers
FROM Customers
WHERE id not in  (
    SELECT c.id
    FROM Customers c
    LEFT JOIN Orders o ON c.id = o.Customerid
    WHERE o.Customerid IS NOT NULL
);


### 1. Rank() over(partition by [col] order by [col]) 2. Delete from

In [ ]:
With t1 as
(
 Select *,
    rank() over(partition by email order by id desc) as rk
    from person
)

Delete from person
where id in (Select t1.id from t1 where t1.rk>1);

### Avg

In [ ]:
Select query_name, round(sum(rating/position)/count(*),2) as quality, 
round(avg(case when rating<3 then 1 else 0 end)*100,2) as poor_query_percentage
from queries
group by query_name;

### 1. Join (with the same table to compare the next day's temperature) 2. INTERVAL (datetime) 3. Add one day with PostgreSQL (YEAR, MONTH ,DAY ,HOUR ,MINUTE ,SECOND)

In [ ]:
--MySQL (date_add) use Postgresql since it is more intuitive
select a.Id
from weather a
join weather b on a.record_date = date_add(b.record_date, INTERVAL 1 day)
where a.temperature > b.temperature

--PostgreSQL (+)
SELECT a.Id
FROM weather a
JOIN weather b ON a.record_date = b.record_date + INTERVAL '1 day'
WHERE a.temperature > b.temperature;

### Where [col] In and Not In (filtering)

In [ ]:
Select distinct a.buyer_id
from sales a join
product b
on a.product_id = b.product_id
where a.buyer_id in
(Select a.buyer_id from sales a join product b on a.product_id = b.product_id where b.product_name = 'S8') 
and
a.buyer_id not in (Select a.buyer_id from sales a join product b on a.product_id = b.product_id where b.product_name = 'iPhone');


### Where [date col] >= 'YYYY-MM-DD' (filtering datetime)
### Format for datetime: 'HH:MM:SS' or 'YYYY-MM-DD HH:MM:SS'

In [ ]:
select distinct a.product_id, product_name from sales a join product b on a.product_id = b.product_id where a.product_id 
in
(select product_id from sales where sale_date >= '2019-01-01' and sale_date <= '2019-03-31')
and
a.product_id not in 
(select product_id from sales where sale_date > '2019-03-31' or sale_date < '2019-01-01');


### Get the second large

In [ ]:
select max(salary) as SecondHighestSalary
from employee
where salary ! = (Select max(salary)
                   from employee);

### DATEDIFF (this is only for date not time) / Date_part('day') with subtraction

In [2]:
--- MySQL (Datediff) use Postgresql more comprehensive
with t1 as (
select id,login_date,
lead(login_date,4) over(partition by id order by login_date) date_5
from (select distinct * from Logins) b
)

select distinct a.id, a.name from t1
inner join accounts a 
on t1.id = a.id
where DATEDIFF(t1.date_5, login_date) = 4
order by id

--- PostgreSQL (DATE_PART with subtraction)
WITH t1 AS (
    SELECT id, login_date,
           LEAD(login_date, 4) OVER (PARTITION BY id ORDER BY login_date) AS date_5
    FROM (SELECT DISTINCT * FROM Logins) b
)

SELECT DISTINCT a.id, a.name
FROM t1
INNER JOIN accounts a ON t1.id = a.id
WHERE DATE_PART('day', t1.date_5 - login_date) = 4
ORDER BY id;

SyntaxError: invalid syntax (2030557356.py, line 1)

### 1. Multiple Case When 2. max([col]) over(partition by [col]) 

In [ ]:
with t1 as (
select company_id, employee_id, employee_name, salary as sa, 
max(salary) over(partition by company_id) as maximum
from salaries)

select company_id, employee_id, employee_name,
case when t1.maximum<1000 then t1.sa
when t1.maximum between 1000 and 10000 then round(t1.sa*.76,0)
else round(t1.sa*.51,0)
end as salary
from t1;


### row_number() over() (this will assign index based on row number not value of the order col like rank)

In [ ]:
select min(log_id) as start_id, max(log_id) as end_id
from(
select log_id, log_id, row_number() over (order by log_id desc) as rk
from logs) a
group by rk;

### 1. Concat 2. Upper 3. Substring([col], start_idx, end_idx)

In [ ]:
select concat(name,'(',upper(substring(occupation,1,1)),')') 
from occupations
order by name;

### Dense_rank() (will not skip rank if same tie)

In [ ]:
select distinct a.salary
      from
      (select salary, 
      dense_rank() over(order by salary desc) as rk
      from Employee) a
      where a.rk = N

### Date format 'YYYY-MM-DD'to 'YYYY-MM'

In [ ]:
--- For PostgreSQL (use this)
to_char(column_name, 'YYYY-MM')

--- For MySQL
DATE_FORMAT(column_name, '%Y-%m')

### Important!!! Joining the same table to find parent node (ex. manager of employee)

In [ ]:
---Write an SQL query to find number of comments per each post.
---https://leetcode.ca/2019-04-24-1241-Number-of-Comments-per-Post/
---Submissions table:
+---------+------------+
| sub_id  | parent_id  |
+---------+------------+
| 1       | Null       |
| 2       | Null       |
| 1       | Null       |
| 12      | Null       |
| 3       | 1          |
| 5       | 2          |
| 3       | 1          |
| 4       | 1          |
| 9       | 1          |
| 10      | 2          |
| 6       | 7          |
+---------+------------+

---Result table:
+---------+--------------------+
| post_id | number_of_comments |
+---------+--------------------+
| 1       | 3                  |
| 2       | 2                  |
| 12      | 0                  |
+---------+--------------------+


-- There is no primary key for this table, it may have duplicate rows.
-- Each row can be a post or comment on the post.
-- parent_id is null for posts.
-- parent_id for comments is sub_id for another post in the table.

select s_posts.sub_id as post_id,count(distinct s.sub_id) as number_of_comments
from Submissions s
right outer join
(select distinct sub_id from Submissions
where parent_id is null) as s_posts
on s.parent_id = s_posts.sub_id
group by post_id
order by post_id

--- More Example:
SELECT
  e.FirstName + ' ' + e.LastName Employee
  ,m.FirstName + ' ' + m.LastName Manager
FROM Employee e
INNER JOIN Employee m
	ON m.EmployeeID = e.ManagerID
ORDER BY Manager;
                                    

### Min(Case When [col] = 'value') or Max both works(to pivot table)

In [ ]:
SELECT
	Users.id as id,
	Users.username as username,
	MIN(CASE WHEN Users_meta.meta = 'first_name' THEN Users_meta.value ELSE NULL END) AS first_name,
	MIN(CASE WHEN Users_meta.meta = 'last_name' THEN Users_meta.value ELSE NULL END) AS last_name,
	MIN(CASE WHEN Users_meta.meta = 'user_level' THEN Users_meta.value ELSE NULL END) AS user_level
FROM
	Users
LEFT JOIN Users_meta
	ON Users_meta.[user] = Users.id

GROUP BY
	Users.id, users.username;

--- More Example:
<--- 
+----+------+-------+
| ID | Type | Value |
+----+------+-------+
| 1  | A    | 10    |
| 1  | B    | 20    |
| 2  | A    | 30    |
| 2  | B    | 40    |
+----+------+-------+

without Group By and Max, only Case When
+----+---------+---------+
| ID | A_Value | B_Value |
+----+---------+---------+
| 1  |   10    |   NULL  |
| 1  |   NULL  |   20    |
| 2  |   30    |   NULL  |
| 2  |   NULL  |   40    |
+----+---------+---------+

with group by and Max 
+----+---------+---------+
| ID | A_Value | B_Value |
+----+---------+---------+
| 1  | 10      | 20      |
| 2  | 30      | 40      |
+----+---------+---------+ 
-->

-- need Max() and group by here since there is null value 
SELECT
  ID,
  MAX(CASE WHEN Type = 'A' THEN Value END) AS A_Value,
  MAX(CASE WHEN Type = 'B' THEN Value END) AS B_Value
FROM
  your_table_name
GROUP BY
  ID;


### 1. Max(Case When ...) 2. is not NULL

In [ ]:
WITH t1 as (
SELECT candidate_id,
      MAX(CASE WHEN skill = 'Python' THEN 'yes' END) as python,
      MAX(CASE WHEN skill = 'Tableau' THEN 'yes' END) as tableau,
      MAX(CASE WHEN skill = 'PostgreSQL' THEN 'yes' END) as postgresql
FROM candidates
GROUP BY candidate_id)

SELECT candidate_id
FROM t1
WHERE python is not NULL
AND tableau is not NULL
AND postgresql is not NULL
ORDER BY candidate_id ASC;

### Date Calculation in PostgreSql (:: is only used in postgresql)

In [ ]:
SELECT 
	user_id, 
    MAX(post_date::DATE) - MIN(post_date::DATE) AS days_between
FROM posts
WHERE DATE_PART('year', post_date::DATE) = 2021 
GROUP BY user_id
HAVING COUNT(post_id)>1;

### Date Calculation with INTERVAL

In [ ]:
with t1 as 
(SELECT 
  policy_holder_id,
  call_received AS current_call,
  lag(call_received) OVER(PARTITION BY policy_holder_id ORDER BY call_received) as pre_call
FROM callers)

SELECT count(DISTINCT policy_holder_id) AS patient_count
FROM t1
WHERE current_call - pre_call < INTERVAL '7 days'
;

### Find Duplicates with Group By

In [ ]:
WITH job_count_cte AS (
  SELECT 
    company_id, 
    title, 
    description, 
    COUNT(job_id) AS job_count
  FROM job_listings
  GROUP BY company_id, title, description
)

SELECT COUNT(DISTINCT company_id) AS duplicate_companies
FROM job_count_cte
WHERE job_count > 1;

### Concat Number and symbol or character

In [ ]:
SELECT 
  manufacturer, 
  CONCAT('$', ROUND(SUM(total_sales) / 1000000), ' million') AS sales_mil
FROM pharmacy_sales 
GROUP BY manufacturer;

### Row_number() over(Partition_by ... Order_by ...)

In [ ]:
with t1 as (
SELECT *, 
  row_number() OVER(
  PARTITION BY user_id ORDER BY transaction_date) as r
FROM transactions)

SELECT user_id, spend, transaction_date
FROM t1
WHERE r = 3;

### Using average with "partition by" and "rows between" to get rolling average

In [ ]:
SELECT    
  user_id,    
  tweet_date,   
  ROUND(AVG(tweet_count) OVER (
    PARTITION BY user_id     
    ORDER BY tweet_date     
    ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)
  ,2) AS rolling_avg_3d
FROM tweets;

### Getting the top two value

In [ ]:
with t as (
SELECT category, product, SUM(spend) as total_spend,
      RANK() OVER(PARTITION BY category ORDER BY SUM(spend) DESC) as ranking
FROM product_spend
WHERE date_part('year', transaction_date) = 2022
GROUP BY category, product)

SELECT category, product, total_spend
FROM t
WHERE ranking <= 2
ORDER BY category, ranking;

### Dense_rank() and join lots of tables

In [ ]:
with t1 as (
    SELECT a.artist_name,  
        dense_rank() OVER(ORDER BY COUNT(*) DESC) as artist_rank
    FROM global_song_rank g 
    JOIN songs s on g.song_id = s.song_id
    JOIN artists a on s.artist_id = a.artist_id
    where g.rank <= 10
    GROUP BY a.artist_name)

SELECT * 
FROM t1
WHERE artist_rank <= 5
;

### Left join 2 times to get caller and receiver

In [ ]:
with t1 as (
    SELECT pc.caller_id, pc.receiver_id, 
        pic.country_id as caller_country, 
        pir.country_id as receiver_country
    FROM phone_calls pc 
    LEFT JOIN phone_info pic on pc.caller_id = pic.caller_id
    LEFT JOIN phone_info pir on pc.receiver_id = pir.caller_id)

SELECT ROUND(COUNT(*)*100.0/(SELECT COUNT(*) FROM t1),1)
FROM t1
WHERE caller_country != receiver_country
;

### Create 2 tables for current month and previous month with lag([col], [val]) over(partition by ... order by ...) 

In [9]:
-- note here: lag is based on the rows, and distinct is based on the whole combination of rows

WITH a AS (
select distinct user_id, date_part('month', event_date) as aaa
from user_actions
),

b AS (
SELECT a.aaa as cur_mth,
lag(aaa, 1) OVER (PARTITION BY user_id ORDER BY a.aaa ASC) as pre_mth
FROM a)

SELECT cur_mth as mth,
COUNT(*) as count
FROM b
WHERE cur_mth - pre_mth = 1
AND cur_mth = 7
GROUP BY cur_mth
;

SyntaxError: invalid syntax (2954275414.py, line 1)

### 1. Median: Percentile_cont(0.5) (postgresql only) 2. Create number of rows based on frequency using recursive method

In [ ]:
-- flatten the table
with recursive nums as (
    SELECT 
        1 as value, 
        searches,
        num_users
    FROM search_frequency
    union all 
    SELECT 
        value + 1 as value,
        searches,
        num_users
    WHERE value < num_users
)
-- find median (for postgresql)
select round(percentile_cont(0.5) within group(order by searches::decimal), 1) as median
FROM nums

### create number from 1 to 5 (recursive method)

In [ ]:
WITH RECURSIVE numbers AS (
  SELECT 1 AS n
  UNION ALL
  SELECT n + 1 FROM numbers WHERE n < 5 -- Adjust the end value as needed
)
SELECT n FROM numbers;

### Cross join to make combination of the same table

In [ ]:
-- where statement make the combination unique
SELECT 
  concat(p1.topping_name, ',', p2.topping_name, ',', p3.topping_name) as pizza,
  p1.ingredient_cost + p2.ingredient_cost + p3.ingredient_cost as total_cost
FROM pizza_toppings AS p1
cross join
  pizza_toppings as p2,
  pizza_toppings as p3
where p1.topping_name < p2.topping_name
AND p2.topping_name < p3.topping_name
ORDER BY total_cost DESC, pizza ASC;

### RegexP_Like() in where statement

In [ ]:
SELECT name
FROM employees
WHERE REGEXP_LIKE(name, '^J.*');

### Nested case when (if statement) to exchange seat

In [ ]:
SELECT 
    CASE
        WHEN id < (SELECT MAX(id) FROM seat) THEN
            CASE
                WHEN id % 2 = 0 THEN id - 1
                ELSE id + 1
            END
        ELSE
            CASE
                WHEN id % 2 = 0 THEN id - 1
                ELSE id
            END
    END AS id,
    student
FROM seat
ORDER BY id;


# Great advanced practices

### Retrieve top 5 highest salary employee in each department

In [ ]:
with t1 as (
    select 
        department,
        employee,
        salary,
        row_number() over (partition by department order by salary desc) as rank
    from employees
)

select
    department,
    employee,
    salary
from t1
where rank <= 5;

### Calculate total sales for each month of the current year (including months with zero sales)
### This is for postgresql (generate_series(), date_trunc(), current_date), otherwise you need to use recursive for mysql

  sale_date  
------------
 2023-01-01
 2023-02-01
 2023-03-01
 2023-04-01
 2023-05-01
 2023-06-01
 2023-07-01
 2023-08-01
 2023-09-01
 2023-10-01
 2023-11-01
 2023-12-01
(12 rows)


In [ ]:
-- this cte is to generate the month table that has all the month
with t1 as (
    generate_series(
        date_trunc('year', current_date), -- from when (beginning of the year)
        date_trunc('year', current_date) + interval '1 year' - interval '1 day', -- to when (end of the year)
        interval '1 month' -- interval
    ) as month_date
)

select 
    to_char(t1.month_date, 'YYYY-MM') as month,
    coalesce(sum(sales.sales), 0) as monthly sales
from 
    t1 
left join sales on to_char(t1.month_date, 'YYYY-MM') = to_char(sales.sales_date, 'YYYY-MM')
group by month;

### Find customers who have made a purchase every month for last six months
### date_trunc() only used in postgresql
### DATE_FORMAT(your_date_column, '%Y-%m-01') is the version in MySQL

In [3]:
--- date_trunc vs. date_part
--- https://stackoverflow.com/questions/75389458/when-to-use-date-trunc-vs-date-part

select 
    customer_id
from
    purchase
where
    date_trunc('month', purchase_date) - interval '1 month' <= 6
group by 
    customer_id
having
    count(distinct date_trunc('month', purchase_date)) = 6;

SyntaxError: invalid syntax (1943954009.py, line 1)

### Calculate running total of sales for each day within the past month
### 1. generate_series() is only in postgresql
### 2. generate_series() is same as using recursive call in mysql

In [ ]:
-- use cte to create date table
with t1 as (
    generate_series(
        date_trunc('month', current_date) - interval '1 month', -- from when
        date_trunc('month', current_date) - interval '1 day', -- to when (inclusive)
        interval '1 day' -- interval
    ) as date
)
select
    t1.date,
    sum(coalesce(sales.sales_amount, 0)) over(order by t1.date) as running total
from 
    t1
left join sales on t1.date = sales.date;

### Find the employees who manage the same number of employees as their manager

In [ ]:
select
    e1.employee_name as employee,
    e1.managed_count as managed_count
from 
    employees e1
join 
    employees e2 on e1.manger_id = e2.employee_id
where 
    e1.managed_count = e2.managed_count;

### calculate 30-day moving average of sales for each product (range between)

In [ ]:
select
    product_id,
    sales_date,
    sales_amount,
    avg(sales_amount) over (
                    partition by product_id 
                    order by sales_date 
                    range between interval '30 days' preceding and current row) as moving_avg
from
    sales;